<div>
<img src="https://www.nebrija.com/images/logos/logotipo-universidad-nebrija.jpg" width="200">
</div>

**MODELOS DE PROGRAMACION: MODELO ADIABATICO** -
Prof: Carmen Pellicer Lostao

# CSP DIET PLANNING

## EXERCICE

In this exercice you have to implement a **DIET PLANNER** application following this  example: https://docs.ocean.dwavesys.com/en/stable/examples/hybrid_cqm_diet.html in the examples repository of DWave's OCEAN SDK

The example will guide you on the code to build a **Constrained Quadratic Model (CQM)** and solve it in the **Leap hybrid Constrained Quadratic Models (CQM) solver**

### REQUIEREMENTS FOR YOUR DIET PLANNER

The Diet Planner that you must develop will have:

- a list of your own foods
- your own constraints on bound values of the foods

Given the above own diet you must:

- solve the problem in the LeapHybridCQMSampler
- tune the result to find good values for the parameters that scale the two parts of the optimizing function (taste and cost)

### Additional remarks

Document your notebook describing with text cells all assumptions and definitions that you are using, as well as comment your code cells accordingly.

### DIET PLANNER

In [4]:
import dimod
from dwave.system import LeapHybridCQMSampler
import numpy as np
import matplotlib
matplotlib.use("agg")
import matplotlib.pyplot as plt

In [5]:
# Definimos el diccionario de alimentos
foods = {
  'chicken': {'Calories': 239, 'Protein': 27, 'Fat': 13, 'Carbs': 0, 'Fiber': 0,
              'Taste': 8, 'Cost': 3.0, 'Units': 'continuous'},
  'salmon': {'Calories': 206, 'Protein': 22, 'Fat': 13, 'Carbs': 0, 'Fiber': 0,
             'Taste': 9, 'Cost': 5.5, 'Units': 'continuous'},
  'spinach': {'Calories': 23, 'Protein': 2.9, 'Fat': 0.4, 'Carbs': 3.6, 'Fiber': 2.2,
              'Taste': 6, 'Cost': 1.2, 'Units': 'continuous'},
  'almonds': {'Calories': 579, 'Protein': 21, 'Fat': 50, 'Carbs': 22, 'Fiber': 12,
              'Taste': 7, 'Cost': 4.8, 'Units': 'discrete'},
  'apple': {'Calories': 95, 'Protein': 0.5, 'Fat': 0.3, 'Carbs': 25, 'Fiber': 4.4,
            'Taste': 9, 'Cost': 0.8, 'Units': 'discrete'},
  'yogurt': {'Calories': 149, 'Protein': 5, 'Fat': 8, 'Carbs': 11, 'Fiber': 0,
             'Taste': 6, 'Cost': 2.3, 'Units': 'continuous'}
}
min_nutrients = {"Protein": 50, "Fat": 30, "Carbs": 130, "Fiber": 30}
max_calories = 2000

In [6]:
quantities = [dimod.Real(f"{food}") if foods[food]["Units"] == "continuous"
                                    else dimod.Integer(f"{food}")
                                    for food in foods.keys()]

# Objective Function

In [ ]:
cqm = dimod.ConstrainedQuadraticModel()

In [ ]:
def total_mix(quantity, category):
  return sum(q * c for q, c in zip(quantity, (foods[food][category] for food in foods.keys())))

In [ ]:
cqm.set_objective(-total_mix(quantities, "Taste") + 6*total_mix(quantities, "Cost"))

# Constraints

cqm.add_constraint(total_mix(quantities, "Calories") <= max_calories, label="Calories")
'Calories'

In [ ]:
for nutrient, amount in min_nutrients.items():
  cqm.add_constraint(total_mix(quantities, nutrient) >= amount, label=nutrient)

In [ ]:
list(cqm.constraints.keys())
print(cqm.constraints["Calories"].to_polystring())
print(cqm.constraints["Protein"].to_polystring())

# Solve the Problem by Sampling

In [ ]:
from dwave.system import LeapHybridCQMSampler
sampler = LeapHybridCQMSampler()       

In [ ]:
sampleset = sampler.sample_cqm(cqm)                    
feasible_sampleset = sampleset.filter(lambda row: row.is_feasible)   
print("{} feasible solutions of {}.".format(len(feasible_sampleset), len(sampleset)))    

In [ ]:
def print_diet(sample):
   diet = {food: round(quantity, 1) for food, quantity in sample.items()}
   print(f"Diet: {diet}")
   taste_total = sum(foods[food]["Taste"] * amount for food, amount in sample.items())
   cost_total = sum(foods[food]["Cost"] * amount for food, amount in sample.items())
   print(f"Total taste of {round(taste_total, 2)} at cost {round(cost_total, 2)}")
   for constraint in cqm.iter_constraint_data(sample):
      print(f"{constraint.label} (nominal: {constraint.rhs_energy}): {round(constraint.lhs_energy)}")

In [ ]:
best = feasible_sampleset.first.sample                       
print_diet(best)   

# Tuning the Solution

In [ ]:
cqm.set_objective(-total_mix(quantities, "Taste"))
sampleset_taste = sampler.sample_cqm(cqm)                     
feasible_sampleset_taste = sampleset_taste.filter(lambda row: row.is_feasible)  
best_taste = feasible_sampleset_taste.first                   
print(round(best_taste.energy)) 

In [ ]:
cqm.set_objective(total_mix(quantities, "Cost"))
sampleset_cost = sampler.sample_cqm(cqm)                     
feasible_sampleset_cost = sampleset_cost.filter(lambda row: row.is_feasible)  
best_cost = feasible_sampleset_cost.first                    
print(round(best_cost.energy))                               
print_diet(best_cost.sample)

In [ ]:
cqm.set_objective(-total_mix(quantities, "Taste") + 1 * total_mix(quantities, "Cost"))
sampleset = sampler.sample_cqm(cqm)                        
feasible_sampleset = sampleset.filter(lambda row: row.is_feasible)  
best = feasible_sampleset.first.sample                     
print_diet(best)      

In [ ]:
cqm.set_objective(-total_mix(quantities, "Taste") + 6*total_mix(quantities, "Cost"))
for variable in cqm.variables:
   cqm.set_lower_bound(variable, 1)
sampleset_diverse = sampler.sample_cqm(cqm)                   
feasible_sampleset_diverse = sampleset_diverse.filter(lambda row: row.is_feasible)  
best_diverse = feasible_sampleset_diverse.first.sample        
print_diet(best_diverse) 